In [1]:
import scanpy as sc

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [30]:
bdata = sc.read_h5ad("/lustre/groups/ml01/workspace/alejandro.tejada/COLE/fullDataZebra.h5ad")

In [31]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/zebrafish_processed.h5ad")

In [32]:
bdata = bdata[adata.obs_names]

In [33]:
bdata.n_obs

2686684

In [34]:
bdata.obs = adata[bdata.obs_names].obs

In [35]:
bdata.obs["gene_target_1"] = bdata.obs["gene_target_1"].replace('control', 'ctrl')
bdata.obs["gene_target_2"] = bdata.obs["gene_target_2"].replace('control', 'ctrl')


/tmp/ipykernel_4091174/2827830222.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  bdata.obs["gene_target_1"] = bdata.obs["gene_target_1"].replace('control', 'ctrl')
/tmp/ipykernel_4091174/2827830222.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  bdata.obs["gene_target_1"] = bdata.obs["gene_target_1"].replace('control', 'ctrl')
/tmp/ipykernel_4091174/2827830222.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  bdata.obs["gene_target_2"] = bdata.obs["gene_target_2"].replace('control', 'ctrl')
/tmp/i

In [36]:
adata.obs["gene_target_1"].value_counts()

gene_target_1
control    610839
tfap2a     286482
tbx16      187545
mafba      160550
hgfa       149594
noto       145401
epha4a     124676
egr2b      118279
foxd3      116680
cdx4       105868
met         91860
tbx1        87439
foxi1       81673
zc4h2       73264
hand2       72204
hoxb1a      71278
phox2a      71086
smo         57617
tbxta       39671
wnt3a       34678
Name: count, dtype: int64

In [37]:
adata.obs["gene_target_2"].value_counts()

gene_target_2
control    2358321
foxd3       127133
msgn1        60275
tbx16l       58011
cdx1a        48266
wnt8         34678
Name: count, dtype: int64

In [38]:
bdata.obs["condition"] = bdata.obs.apply(lambda x: x["gene_target_1"] + "+" + x["gene_target_2"], axis=1)

/tmp/ipykernel_4091174/1246144728.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  bdata.obs["condition"] = bdata.obs.apply(lambda x: x["gene_target_1"] + "+" + x["gene_target_2"], axis=1)


In [39]:
bdata.obs["condition"].value_counts()

condition
control_control_48    204327
control_control_36    158564
control_control_72    115985
control_control_24     80761
hgfa_control_72        77277
                       ...  
tbxta_control_18       11681
wnt3a_wnt8_18          11184
cdx4_cdx1a_24           9311
tbxta_control_24        8684
wnt3a_wnt8_24           4736
Name: count, Length: 76, dtype: int64

In [40]:
bdata.var["gene_name"] = bdata.var["gene_short_name"]

In [41]:
import numpy as np
# check if normalized
np.unique(bdata.X[1].A)

array([ 0.,  1.,  2.,  3.,  4.,  5., 13., 14., 29.], dtype=float32)

In [42]:
sc.pp.normalize_total(bdata)
sc.pp.log1p(bdata)
sc.pp.highly_variable_genes(bdata,n_top_genes=5000, subset=False)

In [43]:
genes_to_keep =set(bdata.obs["gene_target_1"].unique()).union(set(bdata.obs["gene_target_2"].unique())) - set("ctrl",)

In [44]:
id_to_keep = set(bdata.var[bdata.var["gene_short_name"].isin(genes_to_keep)].index)

In [45]:
hvgs = bdata.var[bdata.var["highly_variable"]].index

In [46]:
filter_var= list(hvgs)+ list(id_to_keep)

In [47]:
bdata = bdata[:, filter_var]

In [48]:
bdata.obs["cell_type"] = "zebrafish" # we don't have different cell lines

/tmp/ipykernel_4091174/211132322.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  bdata.obs["cell_type"] = "zebrafish" # we don't have different cell lines
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [49]:
bdata.write_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/zebrafish_processed_for_gears.h5ad")

In [50]:
bdata_vals = bdata.var.gene_short_name.values

In [51]:
for gene in bdata.obs["gene_target_1"].unique():
    if gene not in bdata_vals:
        print("not found", gene)
    else:
        print("found", gene)

not found control
found zc4h2
found met
found tfap2a
found hgfa
found noto
found cdx4
found epha4a
found mafba
found tbx16
found smo
found hand2
found egr2b
found foxi1
found hoxb1a
found tbx1
found wnt3a
found phox2a
found foxd3
found tbxta
